# Oracle Machine Learning for Python - Introduction
Oracle Machine Learning for Python (OML4Py), a component of the Oracle Advanced Analytics option to Oracle Database Enterprise Edition, makes the open source Python scripting language and environment ready for the enterprise and big data. Designed for problems involving both large and small volumes of data, Oracle Machine Learning for Python integrates Python with Oracle Database, allowing users to execute Python commands and scripts for statistical, machine learning, and graphical analyses on database tables and views using Python syntax. Many familiar Python functions are overloaded and translate Python functions into SQL for in-database execution, as well as new automated machine learning capabilities. 
![title](img/OML4P_icon.jpg)
In this notebook, we highlight a few of the features of OML4Py:
- connect to Oracle Database
- create database tables
- use the transparency layer
- rank attributes for predictive value using the in-database attribute importance algorithm
- build a predictive model
- score data using this model

# Import the OML4Py library and connect to Oracle Database
To use OML4Py, first import the package ***oml***. OML4Py supports a variety of connection specification options, including Oracle Wallet. Once connected to an Oracle Database that has OML4Py installed, invoking ***oml.isconnected*** returns true. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import oml
oml.connect("pyquser","pyquser",
            dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(service_name=ORCLPDB)))')
oml.isconnected()

True

# Create a Pandas DataFrame and load into Oracle Database
We have a couple of options for loading data into Oracle Database. In this first example, we load the _iris_ data and combine target and predictors into a single DataFrame, which matches the form the data would have as a database table. This DataFrame can then be loaded into Oracle Database using the ***push*** function, which creates a temporary table. Such temporary tables will be automatically deleted when the database connection is terminated unless saved in a datastore (covered in another notebook). 

In [2]:
from sklearn.datasets import load_iris
from sklearn import linear_model
import pandas as pd

iris = load_iris()
x = pd.DataFrame(iris.data, 
                 columns = ["SEPAL_LENGTH", "SEPAL_WIDTH", "PETAL_LENGTH", "PETAL_WIDTH"])
y = pd.DataFrame(list(map(lambda x: {0:'setosa', 1: 'versicolor', 2:'virginica'}[x], iris.target)), 
                 columns = ['Species'])
iris_df = pd.concat([x,y], axis=1)

IRIS_TMP = oml.push(iris_df)
print("Shape:",IRIS_TMP.shape)
print("Columns:",IRIS_TMP.columns)
IRIS_TMP.head(4)

Shape: (150, 5)
Columns: ['SEPAL_LENGTH', 'SEPAL_WIDTH', 'PETAL_LENGTH', 'PETAL_WIDTH', 'Species']


   SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH Species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa

We can also create a persistent table using the ***create*** function and specifying a tablename, "IRIS".

In [ ]:
try:
    oml.drop(table="IRIS")
except:
    pass
IRIS = oml.create(iris_df, table="IRIS")
print("Shape:",IRIS.shape)
IRIS.head(4)

See what functions are available on IRIS as an OML DataFrame.

In [ ]:
res = [x for x in IRIS.__dir__() if not x.startswith('_')]
res.sort()
res

Perform a few aggregate operations on the IRIS database table SEPAL_LENGTH column using the transparency layer. 

In [ ]:
print("Add 5 to each SEPAL_LENGTH: ",IRIS["SEPAL_LENGTH"] + 5)
print("\nMin SEPAL_LENGTH: ", IRIS["SEPAL_LENGTH"].min())
print("\nMean SEPAL_LENGTH: ",IRIS["SEPAL_LENGTH"].mean())

Explore masking using IRIS PETAL_LENGTH column.

In [ ]:
print("PETAL_LENGTH: ", IRIS["PETAL_LENGTH"])

MASK1 = IRIS["PETAL_LENGTH"] > 4.0
print("\nPETAL_LENGTH > 4.0: ", IRIS["PETAL_LENGTH"][MASK1])

MASK2 = (IRIS["PETAL_LENGTH"] < 2) | (IRIS["PETAL_LENGTH"] > 5)
print("\nPETAL_LENGTH < 2 | > 5: ", IRIS["PETAL_LENGTH"][MASK2])

# Explore data by computing pairwise correlation 
Using only numeric columns, we can compute a correlation matrix on the database table IRIS using the overloaded ***corr*** function. Here we see that petal length and petal width are highly correlated.

In [ ]:
IRIS.corr()

# Explore distribution of IRIS numeric columns
OML4Py overloads a select graphics functions as well. Here, we illustrate the use of ***boxplot*** to show the distribution of the various columns. Here, the statistical computations take place in the database - avoiding data movement - and returns only the summary statistics needed to produce the plot, which enables scalability.

In [ ]:
import matplotlib.pyplot as plt

oml.graphics.boxplot(IRIS[:, :4], notch=True, showmeans = True, 
                     labels=['Sepal Length', 'Sepal Width','Petal Length', 'Petal Width'])
plt.title('Distribution of IRIS Attributes')
plt.ylabel('cm');

# Compute attribute importance ranking of variables 
Using the IRIS table, determine which variables (a.k.a., attributes or predictors) are most important in predicting target 'Species'.

Since we're starting with a database table, we split IRIS into test and train data sets, and then separate the target from the predictors in the training set.

In the setting variable, we disable sampling. Note that OML4Py provides parameters to algorithms as name-value pairs. See the Oracle Data Mining SQL API for available options. 

Then, we define the attribute importance object, compute attribute importance using function ***fit***, and show the results, which includes the importance ranking.

Notice that petal width is most predictive of the target species. The importance value produced by this algorithms provides a relative ranking, not an absolute number, to be used to distinguish importance among variables.

In [ ]:
from oml import ai

train_dat, test_dat = oml.sync(table = "IRIS").split()  # here we use sync to get handle to existing table
train_x = train_dat.drop('Species')
train_y = train_dat['Species']

setting = {'ODMS_SAMPLING':'ODMS_SAMPLING_DISABLE'}

In [ ]:
ai_obj = ai(**setting)
ai_obj = ai_obj.fit(train_x, train_y)

In [ ]:
ai_obj

# Build predictive models
In this example, we set up the data and a _demo_ cost matrix to build a Random Forest model to predict Species from the IRIS table. 

Note that we create the cost matrix as a named database table, RF_COST. 

In [ ]:
dat = IRIS.split()
train_x = dat[0].drop('Species')
train_y = dat[0]['Species']
test_dat = dat[1]

cost_matrix = [['setosa', 'setosa', 0],
  ['setosa', 'virginica', 0.2],
  ['setosa', 'versicolor', 0.8],
  ['virginica', 'virginica', 0],
  ['virginica', 'setosa', 0.5],
  ['virginica', 'versicolor', 0.5],
  ['versicolor', 'versicolor', 0],
  ['versicolor', 'setosa', 0.4],
  ['versicolor', 'virginica', 0.6]]
try:
    oml.drop(table='RF_COST')
except:
    pass
cost_matrix = oml.create(pd.DataFrame(cost_matrix, 
                                      columns = ['ACTUAL_TARGET_VALUE', 
                                                 'PREDICTED_TARGET_VALUE', 'COST']), 
                         "RF_COST")
cost_matrix.pull().pivot(index='ACTUAL_TARGET_VALUE', 
                         columns='PREDICTED_TARGET_VALUE',values='COST')

Build a Random Forest model specifying a maximum tree depth of 4, and display model details.

Note that the model also reports attribute importance. 

In [ ]:
from oml import rf

rf_mod = rf(tree_term_max_depth = '4')
rf_mod = rf_mod.fit(train_x, train_y, cost_matrix)

rf_mod

# Use the Random Forest model to predict Species
When predicting values, users can include _supplemental columns_ from the original data. This can be particularly useful to compare actual target values against predicted values, or to include unique keys for subsequent row identification. This is necessary since row order is not maintained on results from the database. This is consistent with how SQL and relational databases only provide ordering if requested by the user via a sort. 

In [ ]:
pred = rf_mod.predict(test_dat.drop('Species'), 
                      supplemental_cols = test_dat[:, ['SEPAL_LENGTH', 'SEPAL_WIDTH', 'PETAL_LENGTH', 'Species']])
print("Shape:",pred.shape)
pred.head(10)

# Generate and display a Confusion Matrix for the predicted results
Using the overloaded ***crosstab*** function, compute the confusion matrix between the actual values in Species and the predicted result in the PREDICTION column. 

In [ ]:
res_ct = pred.crosstab('Species','PREDICTION',pivot=True)
print("Type:",type(res_ct))
print("Columns:",res_ct.columns)

Since columns are sorted by name after pivot and the rows are not, sort the rows as a post-processing step.

In [ ]:
res_ct.sort_values(by='Species')

# Analyze a larger data set 
Using another, but significanlty larger data set, ONTIME_S, perform a cross tabulation to determine the number of flights to each destination airport. Then, determine predictor ranking (attribute importance). 

As an exercise for the reader, continue to bulid models and score data to predict arrival delay of flights using target variable ARRDELAY.

In [ ]:
ONTIME_S = oml.sync(table='ONTIME_S')
print("Type:",type(ONTIME_S))
print("Shape:",ONTIME_S.shape)

Perform aggregation by counting the number of flights going to each destination airport. 

Note that result is an OML DataFrame that remains in the database, available for futher processing.

In [ ]:
res = ONTIME_S.crosstab('DEST')
print("Type:",type(res))
print("Shape:",res.shape)
res.head()

Compute attribute importance ranking of variables in ONTIME_S to predict ARRDELAY, then use this result to reduce the data set to the most important attributes to build models that predict ARRDELAY. Then display the attribute importance ranking. 

In [ ]:
dat = oml.sync(table = "ONTIME_S").split()  # could also just use ONTIME_S since already loaded in variable
train_x = dat[0].drop('ARRDELAY')
train_y = dat[0]['ARRDELAY']
test_dat = dat[1]

setting = {'ODMS_SAMPLING':'ODMS_SAMPLING_DISABLE'}

ai_obj2 = ai(**setting)
%time ai_obj2 = ai_obj2.fit(train_x, train_y)

In [ ]:
ai_obj2

In [ ]:
oml.disconnect()

<img src="img/Oracle-sm.jpg">